<a href="https://colab.research.google.com/github/Talizg03/weather/blob/main/update_3_2_25ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
df = pd.read_csv("Train.csv")

<ipython-input-1-97fbf718528a>:2: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Train.csv")


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141777 entries, 0 to 141776
Data columns (total 53 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   SalesID                   141777 non-null  int64  
 1   SalePrice                 141777 non-null  int64  
 2   MachineID                 141777 non-null  int64  
 3   ModelID                   141777 non-null  int64  
 4   datasource                141777 non-null  int64  
 5   auctioneerID              141777 non-null  int64  
 6   YearMade                  141777 non-null  int64  
 7   MachineHoursCurrentMeter  27039 non-null   float64
 8   UsageBand                 24113 non-null   object 
 9   saledate                  141777 non-null  object 
 10  fiModelDesc               141777 non-null  object 
 11  fiBaseModel               141777 non-null  object 
 12  fiSecondaryDesc           101822 non-null  object 
 13  fiModelSeries             11935 non-null   o

In [3]:
df2=df.copy()
df2.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,66000,999089,3157,121,3,2004,68.0,Low,11/16/2006 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,57000,117657,77,121,3,1996,4640.0,Low,3/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,10000,434808,7009,121,3,2001,2838.0,High,2/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1139251,38500,1026470,332,121,3,2001,3486.0,High,5/19/2011 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1139253,11000,1057373,17311,121,3,2007,722.0,Medium,7/23/2009 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Convert 'date_of_sale' to datetime format
df2['date_of_sale'] = pd.to_datetime(df2['saledate'], format='%m/%d/%Y %H:%M', errors='coerce')

# Add AgeOfVehicle column (date of sale year - YearMade)
df2['AgeOfVehicle'] = df2['date_of_sale'].dt.year - df2['YearMade']

# Display the first few rows of the updated DataFrame

df2.head()





,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,date_of_sale,AgeOfVehicle
0,1139246,66000,999089,3157,121,3,2004,68.0,Low,11/16/2006 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional,2006-11-16,2
1,1139248,57000,117657,77,121,3,1996,4640.0,Low,3/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional,2004-03-26,8
2,1139249,10000,434808,7009,121,3,2001,2838.0,High,2/26/2004 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2004-02-26,3
3,1139251,38500,1026470,332,121,3,2001,3486.0,High,5/19/2011 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-05-19,10
4,1139253,11000,1057373,17311,121,3,2007,722.0,Medium,7/23/2009 0:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-07-23,2


In [5]:
import seaborn as sns

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.inspection import permutation_importance
import pandas as pd

# turn all string/category columns into integers

for col in df2.select_dtypes(['object']):
    df2[col] = df[col].astype('category')
df2.info()

for col in df2.select_dtypes(['category']):
    df2[col] = df2[col].cat.codes

df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141777 entries, 0 to 141776
Data columns (total 55 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   SalesID                   141777 non-null  int64         
 1   SalePrice                 141777 non-null  int64         
 2   MachineID                 141777 non-null  int64         
 3   ModelID                   141777 non-null  int64         
 4   datasource                141777 non-null  int64         
 5   auctioneerID              141777 non-null  int64         
 6   YearMade                  141777 non-null  int64         
 7   MachineHoursCurrentMeter  27039 non-null   float64       
 8   UsageBand                 24113 non-null   category      
 9   saledate                  141777 non-null  category      
 10  fiModelDesc               141777 non-null  category      
 11  fiBaseModel               141777 non-null  category      
 12  fi

In [6]:
# create X and y
X = df2.drop(columns='SalePrice')
display(X.head())
y = df2['SalePrice']
display(y)

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,...,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,date_of_sale,AgeOfVehicle
0,1139246,999089,3157,121,3,2004,68.0,1,594,594,...,-1,-1,-1,-1,-1,-1,2,1,2006-11-16,2
1,1139248,117657,77,121,3,1996,4640.0,1,1579,1067,...,-1,-1,-1,-1,-1,-1,2,1,2004-03-26,8
2,1139249,434808,7009,121,3,2001,2838.0,0,1258,218,...,-1,-1,-1,-1,-1,-1,-1,-1,2004-02-26,3
3,1139251,1026470,332,121,3,2001,3486.0,0,2194,1959,...,-1,-1,-1,-1,-1,-1,-1,-1,2011-05-19,10
4,1139253,1057373,17311,121,3,2007,722.0,2,2915,2189,...,-1,-1,-1,-1,-1,-1,-1,-1,2009-07-23,2


,SalePrice
0,66000
1,57000
2,10000
3,38500
4,11000
...,...
141772,9000
141773,11000
141774,13000
141775,15500


In [7]:
# split into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, train_size=0.7)
X_train

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,...,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,date_of_sale,AgeOfVehicle
41780,1282268,1264707,3170,132,2,1000,NaN,-1,1928,660,...,-1,-1,-1,-1,-1,-1,-1,-1,1996-04-27,996
100429,1418766,1362214,7261,132,1,1970,NaN,-1,1032,1033,...,-1,-1,-1,-1,-1,-1,2,1,2002-12-05,32
97625,1413846,753367,3538,132,1,1998,NaN,-1,1268,474,...,-1,-1,-1,-1,-1,-1,-1,-1,2007-02-27,9
94119,1405680,747876,43,132,1,1990,NaN,-1,2472,478,...,-1,-1,-1,-1,-1,-1,-1,-1,1994-06-16,4
85888,1388913,1234850,3362,132,1,1976,NaN,-1,851,69,...,-1,-1,-1,-1,-1,-1,-1,-1,1994-12-13,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110268,1443270,50586,1574,132,8,1998,NaN,-1,2267,1333,...,-1,-1,-1,0,5,5,-1,-1,2004-05-26,6
119879,1461367,532771,7113,132,2,1987,NaN,-1,1188,484,...,-1,-1,-1,-1,-1,-1,-1,-1,2003-02-02,16
103694,1425773,1420238,4154,132,6,1972,NaN,-1,2204,1451,...,-1,-1,-1,0,6,5,-1,-1,1997-05-20,25
131932,1486248,428435,7256,132,2,2000,NaN,-1,904,1021,...,-1,-1,-1,-1,-1,-1,2,1,2006-12-16,6


In [ ]:
# print the RMSE accuracy of the baseline (std dev)
print("RMSE Baseline accuracy:", y_test.std())

RMSE Baseline accuracy: 22932.4005340408


In [9]:
# create X and y
X = df2.drop(columns=['SalePrice','MachineHoursCurrentMeter','date_of_sale'])
display(X.head())
y = df2['SalePrice']
display(y)

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,UsageBand,saledate,fiModelDesc,fiBaseModel,...,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,AgeOfVehicle
0,1139246,999089,3157,121,3,2004,1,594,594,220,...,-1,-1,-1,-1,-1,-1,-1,2,1,2
1,1139248,117657,77,121,3,1996,1,1579,1067,364,...,-1,-1,-1,-1,-1,-1,-1,2,1,8
2,1139249,434808,7009,121,3,2001,0,1258,218,89,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,3
3,1139251,1026470,332,121,3,2001,0,2194,1959,790,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,10
4,1139253,1057373,17311,121,3,2007,2,2915,2189,875,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,2


,SalePrice
0,66000
1,57000
2,10000
3,38500
4,11000
...,...
141772,9000
141773,11000
141774,13000
141775,15500


In [10]:
# split into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, train_size=0.7)
X_train

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,UsageBand,saledate,fiModelDesc,fiBaseModel,...,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,AgeOfVehicle
41780,1282268,1264707,3170,132,2,1000,-1,1928,660,237,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,996
100429,1418766,1362214,7261,132,1,1970,-1,1032,1033,350,...,-1,-1,-1,-1,-1,-1,-1,2,1,32
97625,1413846,753367,3538,132,1,1998,-1,1268,474,178,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,9
94119,1405680,747876,43,132,1,1990,-1,2472,478,178,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,4
85888,1388913,1234850,3362,132,1,1976,-1,851,69,19,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110268,1443270,50586,1574,132,8,1998,-1,2267,1333,469,...,-1,-1,-1,-1,0,5,5,-1,-1,6
119879,1461367,532771,7113,132,2,1987,-1,1188,484,183,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,16
103694,1425773,1420238,4154,132,6,1972,-1,2204,1451,484,...,-1,-1,-1,-1,0,6,5,-1,-1,25
131932,1486248,428435,7256,132,2,2000,-1,904,1021,345,...,-1,-1,-1,-1,-1,-1,-1,2,1,6


In [11]:
# print the RMSE accuracy of the baseline (std dev)
print("RMSE Baseline accuracy:", y_test.std())

RMSE Baseline accuracy: 23201.39592024731


In [12]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100,random_state=42, n_jobs=-1)
model = model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

def RMSE(y_, y_pred_):
    return ((y_ - y_pred_) ** 2).mean() ** 0.5

print(f'Train RMSE: {RMSE(y_train, y_train_pred):.3f}')
print(f'Test RMSE: {RMSE(y_test, y_test_pred):.3f}')

Train RMSE: 2766.501
Test RMSE: 7481.323


In [13]:
# Train the Isolation Forest model
from sklearn.ensemble import IsolationForest
iso_forest = IsolationForest(n_estimators=100, contamination=0.01, random_state=42)
iso_forest.fit(X_train)

# Predict anomaly scores and labels on a copy of X_test
X_test_copy = X_test.copy()
df2['anomaly_score'] = iso_forest.decision_function(X)  # Quantitative weirdness
df2['anomaly'] = iso_forest.predict(X)  # Binary anomaly label
df2['anomaly_label'] = df2['anomaly'].map({1: 'Normal', -1: 'Anomaly'})

In [14]:
df2.sort_values("anomaly_score").sample(20)

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,date_of_sale,AgeOfVehicle,anomaly_score,anomaly,anomaly_label
24672,1251737,15000,1485145,6788,132,1,1985,NaN,-1,3463,...,-1,-1,-1,-1,-1,1991-09-24,6,0.172181,1,Normal
39168,1277925,22500,1441167,22345,132,1,1993,NaN,-1,2398,...,0,5,5,-1,-1,2005-06-01,12,0.134329,1,Normal
119701,1461140,17000,23017,7113,132,6,1987,NaN,-1,1386,...,-1,-1,-1,-1,-1,1996-03-12,9,0.154021,1,Normal
86226,1389497,31000,343793,3883,132,4,1978,NaN,-1,2153,...,-1,-1,-1,2,1,2002-05-16,24,0.156151,1,Normal
93722,1405169,19500,1463070,7110,132,11,1989,NaN,-1,1587,...,-1,-1,-1,-1,-1,1996-03-27,7,0.176057,1,Normal
128429,1476642,28000,1427492,7277,132,1,1980,NaN,-1,1071,...,-1,-1,-1,2,1,2000-12-08,20,0.164870,1,Normal
54082,1301087,13500,1168882,4123,132,8,1971,NaN,-1,593,...,0,4,5,-1,-1,2005-11-16,34,0.174001,1,Normal
76438,1365898,9500,1505208,4122,132,13,1966,NaN,-1,2383,...,0,7,5,-1,-1,1996-05-09,30,0.170315,1,Normal
4445,1154240,37000,1066582,5273,121,3,2004,1554.0,1,1151,...,0,5,5,-1,-1,2011-02-15,7,0.143728,1,Normal
27794,1256892,28500,1546936,6558,132,4,1983,NaN,-1,1850,...,0,5,5,-1,-1,1989-04-21,6,0.133588,1,Normal


In [15]:
df2.anomaly.value_counts()

,count
anomaly,
1,140385
-1,1392


In [16]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141777 entries, 0 to 141776
Data columns (total 58 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   SalesID                   141777 non-null  int64         
 1   SalePrice                 141777 non-null  int64         
 2   MachineID                 141777 non-null  int64         
 3   ModelID                   141777 non-null  int64         
 4   datasource                141777 non-null  int64         
 5   auctioneerID              141777 non-null  int64         
 6   YearMade                  141777 non-null  int64         
 7   MachineHoursCurrentMeter  27039 non-null   float64       
 8   UsageBand                 141777 non-null  int8          
 9   saledate                  141777 non-null  int16         
 10  fiModelDesc               141777 non-null  int16         
 11  fiBaseModel               141777 non-null  int16         
 12  fi

In [17]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

model = RandomForestRegressor()
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print("Test RMSE:", mean_squared_error(y_test, y_test_pred) ** 0.5)

Test RMSE: 7484.482538378986


In [18]:
df3=df2.copy()
df3.head()

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,...,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,date_of_sale,AgeOfVehicle,anomaly_score,anomaly,anomaly_label
0,1139246,66000,999089,3157,121,3,2004,68.0,1,594,...,-1,-1,-1,2,1,2006-11-16,2,0.095472,1,Normal
1,1139248,57000,117657,77,121,3,1996,4640.0,1,1579,...,-1,-1,-1,2,1,2004-03-26,8,0.054946,1,Normal
2,1139249,10000,434808,7009,121,3,2001,2838.0,0,1258,...,-1,-1,-1,-1,-1,2004-02-26,3,0.107816,1,Normal
3,1139251,38500,1026470,332,121,3,2001,3486.0,0,2194,...,-1,-1,-1,-1,-1,2011-05-19,10,0.059816,1,Normal
4,1139253,11000,1057373,17311,121,3,2007,722.0,2,2915,...,-1,-1,-1,-1,-1,2009-07-23,2,0.041633,1,Normal


In [19]:
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt

for col in df3.select_dtypes(['object']):
    df3[col] = df2[col].astype('category')
df3.info()

for col in df3.select_dtypes(['category']):
    df3[col] = df3[col].cat.codes
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141777 entries, 0 to 141776
Data columns (total 58 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   SalesID                   141777 non-null  int64         
 1   SalePrice                 141777 non-null  int64         
 2   MachineID                 141777 non-null  int64         
 3   ModelID                   141777 non-null  int64         
 4   datasource                141777 non-null  int64         
 5   auctioneerID              141777 non-null  int64         
 6   YearMade                  141777 non-null  int64         
 7   MachineHoursCurrentMeter  27039 non-null   float64       
 8   UsageBand                 141777 non-null  int8          
 9   saledate                  141777 non-null  int16         
 10  fiModelDesc               141777 non-null  int16         
 11  fiBaseModel               141777 non-null  int16         
 12  fi

In [20]:
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt

for col in df3.select_dtypes(['object']):
    df3[col] = df2[col].astype('category')


for col in df3.select_dtypes(['category']):
    df3[col] = df3[col].cat.codes
df3 = df3[df2.anomaly==1].copy()
X2 = df3.drop(columns=['SalePrice', 'anomaly_label', 'anomaly', 'anomaly_score','date_of_sale'])
X2 = pd.get_dummies(X2, drop_first=True)
y2 = df3['SalePrice']
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.3, random_state=42)

model2 = RandomForestRegressor()
model2.fit(X2, y2)
y2_test_pred = model2.predict(X2_test)
y2_train_pred = model2.predict(X2_train)
def RMSE(y2_, y2_pred_):
    return ((y2_ - y2_pred_) ** 2).mean() ** 0.5

print(f'Train RMSE: {RMSE(y2_train, y2_train_pred):.3f}')
print(f'Test RMSE: {RMSE(y2_test, y2_test_pred):.3f}')

Train RMSE: 2688.413
Test RMSE: 2687.493


In [21]:
import joblib
model_data2 = [model2, iso_forest]
joblib.dump(model_data2, '/content/3.2.25_final.pkl')  # Save directly to the file

['/content/3.2.25_final.pkl']

In [24]:
import pandas as pd
df_valid = pd.read_csv("Valid.csv")

In [25]:
df_valid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11573 entries, 0 to 11572
Data columns (total 52 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   SalesID                   11573 non-null  int64  
 1   MachineID                 11573 non-null  int64  
 2   ModelID                   11573 non-null  int64  
 3   datasource                11573 non-null  int64  
 4   auctioneerID              11573 non-null  int64  
 5   YearMade                  11573 non-null  int64  
 6   MachineHoursCurrentMeter  4739 non-null   float64
 7   UsageBand                 4031 non-null   object 
 8   saledate                  11573 non-null  object 
 9   fiModelDesc               11573 non-null  object 
 10  fiBaseModel               11573 non-null  object 
 11  fiSecondaryDesc           8037 non-null   object 
 12  fiModelSeries             1759 non-null   object 
 13  fiModelDescriptor         2897 non-null   object 
 14  Produc

In [26]:
# Convert 'date_of_sale' to datetime format
df_valid['date1_of_sale'] = pd.to_datetime(df_valid['saledate'], format='%m/%d/%Y %H:%M', errors='coerce')

# Add AgeOfVehicle column (date of sale year - YearMade)
df_valid['AgeOfVehicle'] = df_valid['date1_of_sale'].dt.year - df_valid['YearMade']

# Display the first few rows of the updated DataFrame

df_valid.head()

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,...,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,date1_of_sale,AgeOfVehicle
0,1222837,902859,1376,121,3,1000,0.0,NaN,1/5/2012 0:00,375L,...,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN,2012-01-05,1012
1,1222839,1048320,36526,121,3,2006,4412.0,Medium,1/5/2012 0:00,TX300LC2,...,None or Unspecified,Yes,Double,NaN,NaN,NaN,NaN,NaN,2012-01-05,6
2,1222841,999308,4587,121,3,2000,10127.0,Medium,1/5/2012 0:00,270LC,...,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN,2012-01-05,12
3,1222843,1062425,1954,121,3,1000,4682.0,Low,1/5/2012 0:00,892DLC,...,None or Unspecified,None or Unspecified,Double,NaN,NaN,NaN,NaN,NaN,2012-01-05,1012
4,1222845,1032841,4701,121,3,2002,8150.0,Medium,1/4/2012 0:00,544H,...,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional,2012-01-04,10


In [27]:
from sklearn.ensemble import IsolationForest
import matplotlib.pyplot as plt

for col in df_valid.select_dtypes(['object']):
    df_valid[col] = df_valid[col].astype('category')

for col in df_valid.select_dtypes(['category']):
    df_valid[col] = df_valid[col].cat.codes
df_valid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11573 entries, 0 to 11572
Data columns (total 54 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   SalesID                   11573 non-null  int64         
 1   MachineID                 11573 non-null  int64         
 2   ModelID                   11573 non-null  int64         
 3   datasource                11573 non-null  int64         
 4   auctioneerID              11573 non-null  int64         
 5   YearMade                  11573 non-null  int64         
 6   MachineHoursCurrentMeter  4739 non-null   float64       
 7   UsageBand                 11573 non-null  int8          
 8   saledate                  11573 non-null  int8          
 9   fiModelDesc               11573 non-null  int16         
 10  fiBaseModel               11573 non-null  int16         
 11  fiSecondaryDesc           11573 non-null  int8          
 12  fiModelSeries     

In [32]:
print(X2.shape)  # Shape of the data you're passing to decision_function
print(df_valid.shape)  # Shape of the DataFrame you're adding the 'anomaly_score' to

(11573, 53)
(11573, 55)


In [33]:
print(X2.columns)  # Columns of X2
print(df_valid.columns)  # Columns of df_valid


Index(['SalesID', 'MachineID', 'ModelID', 'datasource', 'auctioneerID',
       'YearMade', 'MachineHoursCurrentMeter', 'UsageBand', 'saledate',
       'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc', 'fiModelSeries',
       'fiModelDescriptor', 'ProductSize', 'fiProductClassDesc', 'state',
       'ProductGroup', 'ProductGroupDesc', 'Drive_System', 'Enclosure',
       'Forks', 'Pad_Type', 'Ride_Control', 'Stick', 'Transmission',
       'Turbocharged', 'Blade_Extension', 'Blade_Width', 'Enclosure_Type',
       'Engine_Horsepower', 'Hydraulics', 'Pushblock', 'Ripper', 'Scarifier',
       'Tip_Control', 'Tire_Size', 'Coupler', 'Coupler_System',
       'Grouser_Tracks', 'Hydraulics_Flow', 'Track_Type',
       'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb', 'Pattern_Changer',
       'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type', 'Travel_Controls',
       'Differential_Type', 'Steering_Controls', 'AgeOfVehicle'],
      dtype='object')
Index(['SalesID', 'MachineID', 'ModelID', 'datasour

In [46]:
df_valid=df_valid.drop(columns=['date1_of_sale'])
df_valid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11573 entries, 0 to 11572
Data columns (total 54 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   SalesID                   11573 non-null  int64  
 1   MachineID                 11573 non-null  int64  
 2   ModelID                   11573 non-null  int64  
 3   datasource                11573 non-null  int64  
 4   auctioneerID              11573 non-null  int64  
 5   YearMade                  11573 non-null  int64  
 6   MachineHoursCurrentMeter  4739 non-null   float64
 7   UsageBand                 11573 non-null  int8   
 8   saledate                  11573 non-null  int8   
 9   fiModelDesc               11573 non-null  int16  
 10  fiBaseModel               11573 non-null  int16  
 11  fiSecondaryDesc           11573 non-null  int8   
 12  fiModelSeries             11573 non-null  int8   
 13  fiModelDescriptor         11573 non-null  int8   
 14  Produc

In [47]:
# Train the Isolation Forest model
from sklearn.ensemble import IsolationForest
iso_forest = IsolationForest(n_estimators=100, contamination=0.01, random_state=42)
iso_forest.fit(X2_train)

# Predict anomaly scores and labels on a copy of X_test
X2_test_copy = X2_test.copy()
df_valid['anomaly_score'] = iso_forest.decision_function(X2)  # Quantitative weirdness
df_valid['anomaly'] = iso_forest.predict(X2)  # Binary anomaly label
df_valid['anomaly_label'] = df_valid['anomaly'].map({1: 'Normal', -1: 'Anomaly'})

In [49]:
df_valid.anomaly.value_counts()

,count
anomaly,
1,11095
-1,478


In [55]:
df_valid2=df_valid.drop(columns=['anomaly_score','anomaly_label','anomaly'])
df_valid2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11573 entries, 0 to 11572
Data columns (total 53 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   SalesID                   11573 non-null  int64  
 1   MachineID                 11573 non-null  int64  
 2   ModelID                   11573 non-null  int64  
 3   datasource                11573 non-null  int64  
 4   auctioneerID              11573 non-null  int64  
 5   YearMade                  11573 non-null  int64  
 6   MachineHoursCurrentMeter  4739 non-null   float64
 7   UsageBand                 11573 non-null  int8   
 8   saledate                  11573 non-null  int8   
 9   fiModelDesc               11573 non-null  int16  
 10  fiBaseModel               11573 non-null  int16  
 11  fiSecondaryDesc           11573 non-null  int8   
 12  fiModelSeries             11573 non-null  int8   
 13  fiModelDescriptor         11573 non-null  int8   
 14  Produc

In [56]:
import joblib
from sklearn.ensemble import IsolationForest


# Step 1: Load the entire saved model (both Isolation Forest and main model)
model_data2 = joblib.load('/content/3.2.25_final.pkl')

# Access the models by index (not by string key)
model2 = model_data2[0]  # First element in the list (RandomForestRegressor)
isolation_forest = model_data2[1]  # Second element in the list (Isolation Forest model)

# Step 3: Apply Isolation Forest to detect anomalies in new data (new_df)
outliers_new = isolation_forest.predict(df_valid2)

# Step 4: Keep only the normal rows (where prediction is 1)
new_df_clean = df_valid2[outliers_new == 1]

# Step 5: Make predictions using the main model (RandomForestRegressor)
y2_pred_ = model2.predict(new_df_clean)

# Step 6: Optionally, add the predictions to the cleaned dataframe
new_df_clean['y2_pred_'] = y2_pred_

# Step 7: View the predictions
print(new_df_clean.head())


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- MachineHoursCurrentMeter
